In [1]:
'''
narrowing SIU reports from 2017/18 to only those with the following keywords
the list of all the reports to scrape are in an csv file in reports_to_scrape folder downloaded from their site
Ontario - Special Investigations Unit (SIU Annual Report 2021-2022 and Special Investigations Unit (SIU) director’s reports from 2005 to November 2018 | Ontario.ca) 
Firearm injuries
Custody injuries
Other injuries/deaths
Vehicle injuries
Use of force
Unlawful or unnecessary exercise of authority 
Unnecessary force
Unnecessary use of force
Assault
Sexual assault
Harm
Excessive force
Injury
'''

# import libraries
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
# import pages to scrape from the csv file
df = pd.read_csv("reports_to_scrape/on_siu_2018.csv")
df.head()

,Case number,Special Investigations Unit incident date,Special Investigations Unit published date
0,"<a href=""/page/siu-directors-report-case-17-of...",2017-12-30,2018-10-05
1,"<a href=""/page/siu-directors-report-case-17-ov...",2017-10-11,2018-10-02
2,"<a href=""/page/siu-directors-report-case-17-tc...",2017-11-08,2018-10-02
3,"<a href=""/page/siu-directors-report-case-17-of...",2017-10-27,2018-09-26
4,"<a href=""/page/siu-directors-report-case-18-pv...",2018-06-09,2018-09-25


In [3]:
len(df)

301

In [4]:
# base url for page is: https://www.ontario.ca/
baseurl = "https://www.ontario.ca"

In [5]:
# get the link from the case number part of the df

cases = df["Case number"].tolist()
index = 0
for case in cases:
    # print(case)

    soup = BeautifulSoup(case, "html.parser")
    link = soup.find("a")['href']

    # print(link)
    url = baseurl + link
    
    # scrape the first directors report 17-OFD-379
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    # print(soup.find("title"))
    
    # find if the page contains any of the keywords
    contains_words = False
    keywords = ["Firearm injuries", "Custody injuries", "Other injuries/deaths", "Vehicle injuries", "Use of force", 
                "Unlawful or unnecessary exercise of authority Unnecessary force", "Unnecessary use of force",
                "Assault", "Sexual assault", "Harm", "Excessive force", "Injury"]

    for i in keywords:
        results = soup.body.find_all(string=re.compile('.*{0}.*'.format(i)), recursive=True)
        # print('Found the word "{0}" {1} times\n'.format(i, len(results)))
        if len(results) > 0:
            contains_words = True
    
    if contains_words == False:
        df = df.drop(index)
    
    index += 1

In [6]:
df.head

<bound method NDFrame.head of                                            Case number  \
0    <a href="/page/siu-directors-report-case-17-of...   
1    <a href="/page/siu-directors-report-case-17-ov...   
2    <a href="/page/siu-directors-report-case-17-tc...   
3    <a href="/page/siu-directors-report-case-17-of...   
4    <a href="/page/siu-directors-report-case-18-pv...   
..                                                 ...   
294  <a href="/page/siu-directors-report-case-16-oc...   
295  <a href="/page/siu-directors-report-case-16-tc...   
296  <a href="/page/siu-directors-report-case-17-pc...   
298  <a href="/page/siu-directors-report-case-17-ov...   
299  <a href="/page/siu-directors-report-case-16-oc...   

    Special Investigations Unit incident date  \
0                                  2017-12-30   
1                                  2017-10-11   
2                                  2017-11-08   
3                                  2017-10-27   
4                           

In [7]:
len(df)

290

In [8]:
df.to_csv("reports_to_scrape/on_siu_2018_w_keywords.csv")